In [1]:
# install msodbcsql17
%%sh
apt-get install -y unixodbc-dev
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

Reading package lists...
Building dependency tree...
Reading state information...
unixodbc-dev is already the newest version (2.3.9-5ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.
OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:5 https://packages.microsoft.com/ubuntu/22.04/prod jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100   983  100   983    0     0   4385      0 --:--:-- --:--:-- --:--:--  4408
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    88  100    88    0     0    716      0 --:--:-- --:--:-- --:--:--   715
W: https://packages.microsoft.com/ubuntu/22.04/prod/dists/jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide

In [2]:
!pip install pyodbc

In [3]:
import pyodbc
import pandas as pd
import os

server=''
user=''
password=''

# Create a database connection
conn=pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER='+server+';' # Specify the server name and port
    'UID='+user+';'
    'PWD='+password+';'
)
print(conn)

In [4]:
# for each edge type (fac_type1 -> fac_type2), the edge feature is given by 168*1 0 matrix for [weight, weight, ...]
edge_feature_RM_RP=[0 for i in range(168)]
edge_feature_RP_S=[0 for i in range(168)]
edge_feature_A1_C=[0 for i in range(168)]
edge_feature_A2_C=[0 for i in range(168)]
edge_feature_C_RM=[0 for i in range(168)]

In [5]:
filename='input.csv' # edge set file name

## RM 再利用-原料 -> RP 再利用-產品

In [6]:
def RMtoRP(fac_no1, code_no1):
    # 原料使用量
    qstr="""SELECT *
            FROM IMP_3.dbo.Waste_tmp_month_statistics wtms
            WHERE fac_no = '{}'
                AND CodeNo = '{}'
	            AND AppType = 'manu' --使用
	            AND DType = 1 --原料
            ORDER BY AppTime"""
    df=pd.read_sql(qstr.format(fac_no1, code_no1), conn)

    for index, row in df.iterrows():
        year=int(row['AppTime'][:4])
        month=int(row['AppTime'][-2:])

        # time range: 2010/01 ~ 2023/12
        if year<2010 or year>2023:
            continue

        edge_feature_RM_RP[(year-2010)*12+(month-1)]=row['app_qty']

## RP 再利用-產品 -> S 銷售對象

In [7]:
def RPtoS(fac_no1, fac_no2, fac_name2, WasteName):
    # 銷售量
    qstr="""SELECT *
            FROM IMP.dbo.ReUse_ReuProductSale rurps
            WHERE OrganID = '{}'
	            AND (TargetID = '{}' OR TargetName = '{}')
	            AND WasteName LIKE '%{}%'
            ORDER BY YMDate --申報年月"""
    df=pd.read_sql(qstr.format(fac_no1, fac_no2, fac_name2, WasteName), conn)

    for index, row in df.iterrows():
        year=int(row['YMDate'][:4])
        month=int(row['YMDate'][-2:])

        # time range: 2010/01 ~ 2023/12
        if year<2010 or year>2023:
            continue

        edge_feature_RP_S[(year-2010)*12+(month-1)]=row['SaleQty']

## A1 列管 -> C 清除

In [8]:
def A1toC(fac_no1, fac_no2, code_no1):
    # 再利用聯單, 處理後聯單
    qstr="""SELECT SUM(App_Qty), MONTH(Wcle_date), YEAR(Wcle_date)
            FROM (
                SELECT wdw.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                FROM IMP_3.dbo.Waste_Dlist_wre1 wdw
                LEFT JOIN IMP_3.dbo.Waste_Dlist_wre2 wdw2
                    ON wdw.Dlist_no1 = wdw2.Dlist_no1
                WHERE wdw.Dlist_no2 = wdw2.Dlist_no2
                    AND wdw.Dlist_no3 = wdw2.Dlist_no3
                    AND wdw.Dlist_no1 = '{}'
                    AND Cle_no = '{}'
                    AND Waste_no = '{}'
                UNION
                SELECT wdw3.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                FROM IMP_3.dbo.Waste_Dlist_wt1 wdw3
                LEFT JOIN IMP_3.dbo.Waste_Dlist_wt2 wdw4
                    ON wdw3.Dlist_no1 = wdw4.Dlist_no1
                WHERE wdw3.Dlist_no2 = wdw4.Dlist_no2
                    AND wdw3.Dlist_no3 = wdw4.Dlist_no3
                    AND wdw3.Dlist_no1 = '{}'
                    AND Cle_no = '{}'
                    AND Waste_no = '{}') AS SubQuery
            GROUP BY MONTH(Wcle_date), YEAR(Wcle_date) --清運日期"""
    df=pd.read_sql(qstr.format(fac_no1, fac_no2, code_no1, fac_no1, fac_no2, code_no1), conn)

    for index, row in df.iterrows():
        app_qty=row.iloc[0]
        year=int(row.iloc[2])
        month=int(row.iloc[1])

        # time range: 2010/01 ~ 2023/12
        if year<2010 or year>2023:
            continue

        edge_feature_A1_C[(year-2010)*12+(month-1)]=app_qty

## A2 非列管 -> C 清除

In [9]:
def A2toC(fac_no1, fac_no2, code_no1, fac_name1, fac_name2):
    # 營運紀錄
    if fac_name1==fac_name2 and fac_no1==fac_no2: # if both FacID and Cle_No are the same 統一編號, 為自行清除
        qstr="""SELECT SUM(ReuQty), MONTH(Rec_Date), YEAR(Rec_Date)
            FROM (
	            SELECT Reu_No, FacID, Cle_No, Waste_No, ReuQty, Rec_Date, Reu_Date
	            FROM IMP_1.dbo.Waste_ReuMonReportN_Commit wrmrnc
	            WHERE FacID = '{}'
		            AND Cle_No = 'C0000000'
		            AND Waste_No = '{}') AS SubQuery
            GROUP BY MONTH(Rec_Date), YEAR(Rec_Date) --接受處理(再利用)日期"""
        df=pd.read_sql(qstr.format(fac_no1, code_no1), conn)

        for index, row in df.iterrows():
            ReuQty=row.iloc[0]
            year=int(row.iloc[2])
            month=int(row.iloc[1])

            # time range: 2010/01 ~ 2023/12
            if year<2010 or year>2023:
                continue

            edge_feature_A2_C[(year-2010)*12+(month-1)]=ReuQty
    else: # 一般清除
        qstr="""SELECT SUM(ReuQty), MONTH(Rec_Date), YEAR(Rec_Date)
                FROM (
                    SELECT Reu_No, FacID, Cle_No, Waste_No, ReuQty, Rec_Date, Reu_Date
                    FROM IMP_1.dbo.Waste_ReuMonReportN_Commit wrmrnc
                    WHERE FacID = '{}'
                        AND Cle_No = '{}'
                        AND Waste_No = '{}') AS SubQuery
                GROUP BY MONTH(Rec_Date), YEAR(Rec_Date) --接受處理(再利用)日期"""
        df=pd.read_sql(qstr.format(fac_no1, fac_no2, code_no1), conn)

        for index, row in df.iterrows():
            ReuQty=row.iloc[0]
            year=int(row.iloc[2])
            month=int(row.iloc[1])

            # time range: 2010/01 ~ 2023/12
            if year<2010 or year>2023:
                continue

            edge_feature_A2_C[(year-2010)*12+(month-1)]=ReuQty

## C 清除 -> RM 再利用-原料



In [10]:
def CtoRM(fac_no1, fac_no2, code_no1):
    # 再利用聯單, 處理後聯單, 營運紀錄
    if fac_no1==fac_no2: # if Cle_no = Reu_no, 為再利用者清除
        qstr="""SELECT SUM(App_Qty), MONTH(Wcle_date), YEAR(Wcle_date)
                FROM (
                    SELECT wdw.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                    FROM IMP_3.dbo.Waste_Dlist_wre1 wdw
                    LEFT JOIN IMP_3.dbo.Waste_Dlist_wre2 wdw2
                        ON wdw.Dlist_no1 = wdw2.Dlist_no1
                    WHERE wdw.Dlist_no2 = wdw2.Dlist_no2
                        AND wdw.Dlist_no3 = wdw2.Dlist_no3
                        AND Cle_no = 'C1111111'
                        AND Reu_no = '{}'
                        AND Waste_no = '{}'
                    UNION
                    SELECT wdw3.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                    FROM IMP_3.dbo.Waste_Dlist_wt1 wdw3
                    LEFT JOIN IMP_3.dbo.Waste_Dlist_wt2 wdw4
                        ON wdw3.Dlist_no1 = wdw4.Dlist_no1
                    WHERE wdw3.Dlist_no2 = wdw4.Dlist_no2
                        AND wdw3.Dlist_no3 = wdw4.Dlist_no3
                        AND Cle_no = 'C1111111'
                        AND Reu_no = '{}'
                        AND Waste_no = '{}'
                    UNION
                    SELECT FacID AS Dlist_no1, Cle_No AS Cle_no, Reu_No AS Reu_no, Rec_Date AS Wcle_date, Waste_No AS Waste_no, ReuQty AS App_Qty
                    FROM IMP_1.dbo.Waste_ReuMonReportN_Commit wrmrnc
                    WHERE Cle_No = 'C1111111'
                        AND Reu_No = '{}'
                        AND Waste_No = '{}') AS SubQuery
                GROUP BY MONTH(Wcle_date), YEAR(Wcle_date)
                ORDER BY YEAR(Wcle_date), MONTH(Wcle_date)"""
        df=pd.read_sql(qstr.format(fac_no2, code_no1, fac_no2, code_no1, fac_no2, code_no1), conn)

        for index, row in df.iterrows():
            App_Qty=row.iloc[0]
            year=int(row.iloc[2])
            month=int(row.iloc[1])

            # time range: 2010/01 ~ 2023/12
            if year<2010 or year>2023:
                continue

            edge_feature_C_RM[(year-2010)*12+(month-1)]=App_Qty
    else: # 自行清除 or 一般清除
        qstr="""SELECT SUM(App_Qty), MONTH(Wcle_date), YEAR(Wcle_date)
                FROM (
                    SELECT wdw.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                    FROM IMP_3.dbo.Waste_Dlist_wre1 wdw
                    LEFT JOIN IMP_3.dbo.Waste_Dlist_wre2 wdw2
                        ON wdw.Dlist_no1 = wdw2.Dlist_no1
                    WHERE wdw.Dlist_no2 = wdw2.Dlist_no2
                        AND wdw.Dlist_no3 = wdw2.Dlist_no3
                        AND Cle_no = '{}'
                        AND Reu_no = '{}'
                        AND Waste_no = '{}'
                    UNION
                    SELECT wdw3.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                    FROM IMP_3.dbo.Waste_Dlist_wt1 wdw3
                    LEFT JOIN IMP_3.dbo.Waste_Dlist_wt2 wdw4
                        ON wdw3.Dlist_no1 = wdw4.Dlist_no1
                    WHERE wdw3.Dlist_no2 = wdw4.Dlist_no2
                        AND wdw3.Dlist_no3 = wdw4.Dlist_no3
                        AND Cle_no = '{}'
                        AND Reu_no = '{}'
                        AND Waste_no = '{}'
                    UNION
                    SELECT FacID AS Dlist_no1, Cle_No AS Cle_no, Reu_No AS Reu_no, Rec_Date AS Wcle_date, Waste_No AS Waste_no, ReuQty AS App_Qty
                    FROM IMP_1.dbo.Waste_ReuMonReportN_Commit wrmrnc
                    WHERE Cle_No = '{}'
                        AND Reu_No = '{}'
                        AND Waste_No = '{}') AS SubQuery
                GROUP BY MONTH(Wcle_date), YEAR(Wcle_date)
                ORDER BY YEAR(Wcle_date), MONTH(Wcle_date)"""
        df=pd.read_sql(qstr.format(fac_no1, fac_no2, code_no1, fac_no1, fac_no2, code_no1, fac_no1, fac_no2, code_no1), conn)

        if df.empty: # 列管/非列管自行清除
            qstr="""SELECT SUM(App_Qty), MONTH(Wcle_date), YEAR(Wcle_date)
                    FROM (
                        SELECT wdw.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                        FROM IMP_3.dbo.Waste_Dlist_wre1 wdw
                        LEFT JOIN IMP_3.dbo.Waste_Dlist_wre2 wdw2
                            ON wdw.Dlist_no1 = wdw2.Dlist_no1
                        WHERE wdw.Dlist_no2 = wdw2.Dlist_no2
                            AND wdw.Dlist_no3 = wdw2.Dlist_no3
                            AND Cle_no = 'C0000000'
                            AND Reu_no = '{}'
                            AND Waste_no = '{}'
                            AND wdw.Dlist_no1 = '{}'
                        UNION
                        SELECT wdw3.Dlist_no1, Cle_no, Reu_no, Wcle_date, Waste_no, App_Qty
                        FROM IMP_3.dbo.Waste_Dlist_wt1 wdw3
                        LEFT JOIN IMP_3.dbo.Waste_Dlist_wt2 wdw4
                            ON wdw3.Dlist_no1 = wdw4.Dlist_no1
                        WHERE wdw3.Dlist_no2 = wdw4.Dlist_no2
                            AND wdw3.Dlist_no3 = wdw4.Dlist_no3
                            AND Cle_no = 'C0000000'
                            AND Reu_no = '{}'
                            AND Waste_no = '{}'
                            AND wdw3.Dlist_no1 = '{}'
                        UNION
                        SELECT FacID AS Dlist_no1, Cle_No AS Cle_no, Reu_No AS Reu_no, Rec_Date AS Wcle_date, Waste_No AS Waste_no, ReuQty AS App_Qty
                        FROM IMP_1.dbo.Waste_ReuMonReportN_Commit wrmrnc
                        WHERE Cle_No = 'C0000000'
                            AND Reu_No = '{}'
                            AND Waste_No = '{}'
                            AND FacID = '{}') AS SubQuery
                    GROUP BY MONTH(Wcle_date), YEAR(Wcle_date)
                    ORDER BY YEAR(Wcle_date), MONTH(Wcle_date)"""
            df=pd.read_sql(qstr.format(fac_no2, code_no1, fac_no1, fac_no2, code_no1, fac_no1, fac_no2, code_no1, fac_no1), conn)

            for index, row in df.iterrows():
                App_Qty=row.iloc[0]
                year=int(row.iloc[2])
                month=int(row.iloc[1])

                # time range: 2010/01 ~ 2023/12
                if year<2010 or year>2023:
                    continue

                edge_feature_C_RM[(year-2010)*12+(month-1)]=App_Qty
        else: # 一般清除
            for index, row in df.iterrows():
                App_Qty=row.iloc[0]
                year=int(row.iloc[2])
                month=int(row.iloc[1])

                # time range: 2010/01 ~ 2023/12
                if year<2010 or year>2023:
                    continue

                edge_feature_C_RM[(year-2010)*12+(month-1)]=App_Qty

## Main Function

In [11]:
df_edge=pd.read_csv(filename)

edge_feature=[]

waste_no='R-0201' # RP->S要額外設定

for index, row in df_edge.iterrows():
    if row['fac_type1']=='RM' and row['fac_type2']=='RP':
        RMtoRP(row['fac_no1'], row['code_no1'])
        edge_feature.append(edge_feature_RM_RP)
        edge_feature_RM_RP=[0 for i in range(168)] # re-initialize
    elif row['fac_type1']=='RP' and row['fac_type2']=='S':
        RPtoS(row['fac_no1'], row['fac_no2'], row['fac_name2'], waste_no)
        edge_feature.append(edge_feature_RP_S)
        edge_feature_RP_S=[0 for i in range(168)] # re-initialize
    elif row['fac_type1']=='A1' and row['fac_type2']=='C':
        A1toC(row['fac_no1'], row['fac_no2'], row['code_no1'])
        edge_feature.append(edge_feature_A1_C)
        edge_feature_A1_C=[0 for i in range(168)] # re-initialize
    elif row['fac_type1']=='A2' and row['fac_type2']=='C':
        A2toC(row['fac_no1'], row['fac_no2'], row['code_no1'], row['fac_name1'], row['fac_name2'])
        edge_feature.append(edge_feature_A2_C)
        edge_feature_A2_C=[0 for i in range(168)] # re-initialize
    elif row['fac_type1']=='C' and row['fac_type2']=='RM':
        CtoRM(row['fac_no1'], row['fac_no2'], row['code_no1'])
        edge_feature.append(edge_feature_C_RM)
        edge_feature_C_RM=[0 for i in range(168)] # re-initialize
    else:
        print('Invalid fac_type!')

df=pd.DataFrame(edge_feature)

df.to_csv('edge_feature_format.csv', index=False)

<ipython-input-6-438428a32a11>:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(qstr.format(fac_no1, code_no1), conn)
<ipython-input-7-6c7f98d2954e>:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(qstr.format(fac_no1, fac_no2, fac_name2, WasteName), conn)
<ipython-input-8-d4538ee8429e>:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(qstr.format(fac_no1, fac_no2, code_no1, fac_no1, fac_no2, code_no1), conn)
<ipython-input-9-b161137aac9d>:33: UserWarning: pandas only supports S

In [12]:
df

,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,167
0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
RMtoRP('K8200222', 'R-0201') # 連泰, 廢塑膠
RPtoS('H51A9221', '97337016', '慶毅有限公司', 'R-0201') # 加暘, 慶毅, 廢塑膠
A1toC('O17L2584', 'B25A9221', 'R-0201') # 啟碁, 大豐, 廢塑膠
A2toC('12776442', '12776442', 'R-0201', '鱗鑒環保科技有限公司', '鱗鑒環保科技有限公司') # 鱗鑒, 鱗鑒, 廢塑膠
A2toC('23161533', 'A36B2911', 'R-0201', '丸全台昭股份有限公司', '東泰灣有限公司') # 丸全台昭, 東泰灣, 廢塑膠
CtoRM('K8200222', 'K8200222', 'R-0201') # 連泰, 連泰, 廢塑膠
CtoRM('96405802', 'K8200222', 'R-0201') # 澎湖縣政府, 連泰, 廢塑膠
CtoRM('B25A9221', 'H45B6696', 'R-0201') # 大豐, 永茂, 廢塑膠